In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import gc
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

SAMPLE_SIZE = 0.65
#Caso con edad en mean en vez de 200 para los nan unicamente
#nuevos features
#denominacion empresa mean encoding

%matplotlib inline

In [30]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")

avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos_normales = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
avisos_normales = avisos_normales.loc[:,["nombre_area","idaviso","tipo_de_trabajo","nivel_laboral"]]

vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [31]:
avisos.denominacion_empresa

0                                   VENTOR
1                      Wurth Argentina S.A
2                       ELECTRO OUTLET SRL
3                      Wurth Argentina S.A
4                      Wurth Argentina S.A
5                      Wurth Argentina S.A
6        Agencia Oficial Alejandro Arizaga
7                                cerantola
8                        Hochschild Mining
9                               YAMANIL SA
10                   Adecco -Región Office
11                Romasanta y Asociados SA
12                          Bairestron SRL
13                          Directway S.A.
14                     Wurth Argentina S.A
15           SC Johnson & Son de Argentina
16                          Unión Personal
17                            Motomel S.A.
18                          Asoko Tempo SA
19                         TELEDIFUSORA SA
20                         RE/MAX TITANIUM
21                    Grupo Pujol Figueras
22                                  VENTOR
23         

In [32]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,1
1,48375,RwVdKR,1
2,169730,1KjXB,1
3,169730,2AKzxa,1
4,169730,6LJ64,1


In [33]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,3
1,0005E,25
2,000R8,4
3,001XE,12
4,003k9,26


In [34]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,1,20
1,48375,RwVdKR,1,10
2,169730,1KjXB,1,34
3,169730,2AKzxa,1,1
4,169730,6LJ64,1,10


In [35]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [36]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos_normales, on= "idaviso", how = "left")
vistas_final.head()

,idaviso,idpostulante,cantidad,nombre_area,tipo_de_trabajo,nivel_laboral
0,18,BolNL,0.050000,NaN,NaN,NaN
1,48375,RwVdKR,0.100000,NaN,NaN,NaN
2,169730,1KjXB,0.029412,NaN,NaN,NaN
3,169730,2AKzxa,1.000000,NaN,NaN,NaN
4,169730,6LJ64,0.100000,NaN,NaN,NaN


In [37]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones.sample(len(postulaciones)*3//4), ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

C:\Users\Pocho\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


74

In [38]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

126

In [39]:
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [40]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones = pd.merge(postulaciones,avisos_normales, on = "idaviso", how = "left")
postulaciones.drop("nombre_area_y", axis = 1, inplace=True)
postulaciones.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.cantidad = postulaciones.cantidad.fillna(0)
postulaciones.vistas_postulante = postulaciones.vistas_postulante.fillna(0)

postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
3516016,1112261993,EjPZ1b,0,29.000000,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,3.0,0.0,0.000000,0.0,2.0,2.0,False
1251379,1112425713,Bm5loWL,1,21.000000,True,False,False,False,False,False,...,False,Part-time,Junior,145.0,1.0,0.006897,6.0,27.0,27.0,True
6640739,1112328391,DrXZQEZ,0,19.000000,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,15.0,0.0,0.000000,0.0,13.0,13.0,False
5385604,1112372365,2zPvPj1,1,40.000000,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,6.0,0.0,0.000000,0.0,6.0,6.0,False
595436,1111734483,vVjaXRG,0,30.777591,False,False,True,False,False,False,...,True,Full-time,Senior / Semi-Senior,121.0,0.0,0.000000,18.0,103.0,103.0,False
1360013,1112339145,JBeQMvJ,1,32.000000,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,14.0,0.0,0.000000,5.0,12.0,12.0,False
6421175,1112297496,wVpJx0Z,1,40.000000,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,18.0,0.0,0.000000,1.0,16.0,16.0,False
5167136,1112121429,EPXjVN,0,35.000000,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,5.0,0.0,0.000000,0.0,3.0,3.0,False
3360225,1112460307,vVkJLVX,1,24.000000,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,54.0,1.0,0.018519,8.0,38.0,38.0,True
4357667,1112381513,lDbkJo3,1,25.000000,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,17.0,0.0,0.000000,0.0,14.0,14.0,False


In [41]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [42]:
postulaciones.dropna(axis = 1, inplace=True)

In [43]:
postulaciones_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Cadetería,1
1,0z5Dmrd,Call Center,1
2,0z5Dmrd,Comercial,1
3,0z5Dmrd,Ingeniería Eléctrica y Electrónica,1
4,0z5Dmrd,Ingeniería Mecánica,1


In [44]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')

postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.drop(["tipo_de_trabajo","nivel_laboral"], axis = 1, inplace=True)



In [45]:
#mean encoding
area_mean = postulaciones.groupby("nombre_area").agg({"sepostulo":"mean"})
area_mean.head()

means = area_mean.to_dict()["sepostulo"]
postulaciones.nombre_area = postulaciones.nombre_area.replace(means)
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
4703233,1112422091,2zkRdaV,0,19.0,True,False,False,False,False,False,...,22.0,0.0,0.0,5.0,19.0,19.0,False,1,14,12
1842313,1112358408,EzepWEb,0,24.0,True,False,False,False,False,False,...,4.0,0.0,0.0,3.0,1.0,1.0,False,2,5,1
1903609,1112414520,W9AzxGR,0,23.0,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,False,0,9,6
154042,1112263416,8MPzQoO,1,25.0,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,False,3,12,20
6211860,1111926872,eMzl2B,0,30.0,True,False,False,False,False,False,...,9.0,0.0,0.0,0.0,7.0,7.0,False,0,6,2
4862006,1112425064,zvaX03k,1,29.0,False,True,False,False,False,False,...,5.0,1.0,0.2,2.0,5.0,5.0,True,5,7,1
2150611,1112350546,W9Aj30B,0,23.0,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,False,0,39,27
814907,1112342505,2zPEJ2o,1,22.0,True,False,False,False,False,False,...,15.0,0.0,0.0,2.0,9.0,9.0,False,4,8,8
6739868,1112514091,kP6XXQZ,0,26.0,True,False,False,False,False,False,...,14.0,0.0,0.0,0.0,12.0,12.0,False,0,12,10
3499289,1112468972,pzex0Er,0,32.0,False,True,False,False,False,False,...,2.0,0.0,0.0,0.0,2.0,2.0,False,0,4,0


In [46]:
denom_mean = postulaciones.groupby("denominacion_empresa").agg({"sepostulo":"mean"})
means2 = denom_mean.to_dict()["sepostulo"]
postulaciones.denominacion_empresa = postulaciones.denominacion_empresa.replace(means2)
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
4788358,1112301053,Zp0dKZ,0,32.0,False,True,False,False,False,False,...,5.0,0.0,0.0,0.0,3.0,3.0,False,0,4,2
469068,1112033906,NQdOWx,1,27.0,False,True,False,False,False,False,...,43.0,0.0,0.0,6.0,33.0,33.0,False,0,22,17
3349661,1112417969,Yjl2lwP,0,27.0,True,False,False,False,False,False,...,7.0,0.0,0.0,0.0,6.0,6.0,False,0,15,11
5083194,1112458429,dYjjzQX,1,19.0,False,True,False,False,False,False,...,17.0,0.0,0.0,0.0,11.0,11.0,False,4,36,28
4450663,1112407119,JBrDOrr,1,20.0,False,True,False,False,False,False,...,14.0,0.0,0.0,2.0,13.0,13.0,False,4,29,14
4773356,1111524426,PmJYwkJ,0,40.0,True,False,False,False,False,False,...,1.0,0.0,0.0,0.0,1.0,1.0,False,0,4,4
1784706,1111753928,Zxx2A1,1,31.0,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,False,0,8,1
5788071,1112412133,ow5pwd4,1,26.0,False,True,False,False,False,False,...,1.0,0.0,0.0,1.0,1.0,1.0,False,175,473,343
1883653,1112029547,qe2B0rx,1,27.0,False,True,False,False,False,False,...,1.0,0.0,0.0,0.0,1.0,1.0,False,0,12,5
3835295,1112366184,VNrzowm,1,24.0,False,True,False,False,False,False,...,18.0,0.0,0.0,2.0,17.0,17.0,False,0,9,8


In [47]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
#features.remove('denominacion_empresa')
#features.remove("ciudad")
#features.remove("mapacalle")

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  6515514 Test:  723947


42

In [48]:
model = XGBClassifier(objective='binary:logistic',max_depth=18,learning_rate=0.1,subsample=0.8,colsample_bytree=0.8,n_estimators=1200,silent=False,n_jobs=-1,verbose=1)
dt=model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)

[0]	validation_0-error:0.097593
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.092746
[2]	validation_0-error:0.091066
[3]	validation_0-error:0.088685
[4]	validation_0-error:0.087223
[5]	validation_0-error:0.086733
[6]	validation_0-error:0.086109
[7]	validation_0-error:0.085658
[8]	validation_0-error:0.085376
[9]	validation_0-error:0.085302
[10]	validation_0-error:0.085042
[11]	validation_0-error:0.084574
[12]	validation_0-error:0.083368
[13]	validation_0-error:0.082448
[14]	validation_0-error:0.082418
[15]	validation_0-error:0.082025
[16]	validation_0-error:0.081489
[17]	validation_0-error:0.081083
[18]	validation_0-error:0.08058
[19]	validation_0-error:0.080133
[20]	validation_0-error:0.079898
[21]	validation_0-error:0.079601
[22]	validation_0-error:0.07939
[23]	validation_0-error:0.079044
[24]	validation_0-error:0.078945
[25]	validation_0-error:0.078506
[26]	validation_0-error:0.078227
[27]	validation_0-error:0.077746
[28]	validation_0-error

In [49]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=model.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9843994685334587


In [50]:
print("dale mono")

dale mono


In [51]:
joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean7.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean7.pkl']

In [52]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,avisos_normales,on='idaviso',how='inner')
prediccion.drop("nombre_area_y", axis = 1, inplace=True)
prediccion.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)

prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.nombre_area = prediccion.nombre_area.replace(means)
prediccion.denominacion_empresa = prediccion.denominacion_empresa.replace(means2)
prediccion.sample(10)

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
85348,34191,1112407256,6pWrwj,30.000000,False,True,False,False,False,False,...,33.0,0.0,0.000000,False,0.0,12.0,8.0,0.0,29.0,29.0
34687,35188,1112410028,RzrDJ6o,30.777591,False,False,True,False,False,False,...,30.0,1.0,0.033333,True,0.0,0.0,0.0,7.0,5.0,5.0
5071,82858,1112466289,akjV46W,21.000000,False,True,False,False,False,False,...,367.0,1.0,0.002725,True,30.0,132.0,50.0,68.0,242.0,242.0
94661,93974,1112482065,ZXGwdZ,38.000000,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,0.0,5.0,4.0,0.0,0.0,0.0
91699,90574,1112468159,NZmD0L,38.000000,True,False,False,False,False,False,...,10.0,0.0,0.000000,False,1.0,10.0,9.0,0.0,9.0,9.0
12443,4596,1111473085,0zk2Mpa,38.000000,True,False,False,False,False,False,...,30.0,1.0,0.033333,True,3.0,12.0,4.0,4.0,11.0,11.0
8554,75553,1112464131,KB4J94m,37.000000,False,True,False,False,False,False,...,170.0,1.0,0.005882,True,141.0,168.0,116.0,121.0,153.0,153.0
25832,87008,1112466773,owzLbj3,41.000000,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,4.0,9.0,3.0,0.0,0.0,0.0
72498,53687,1112445399,W9OGlGm,31.000000,False,True,False,False,False,False,...,0.0,0.0,0.000000,False,9.0,125.0,37.0,0.0,0.0,0.0
61221,9182,1111820962,eRYRl8,48.000000,False,True,False,False,False,False,...,10.0,0.0,0.000000,False,0.0,6.0,6.0,0.0,0.0,0.0


In [53]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,26.0,0.0,0.0,False,0.0,7.0,1.0,3.0,20.0,20.0
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,4.0,0.0,0.0,False,3.0,30.0,1.0,0.0,3.0,3.0
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,10.0,0.0,0.0,False,1.0,6.0,0.0,1.0,6.0,6.0
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,0.0,False,0.0,3.0,1.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,1.0,0.0,0.0,False,1.0,4.0,0.0,0.0,1.0,1.0


In [54]:
x_final=prediccion[features]
y_final=model.predict_proba(x_final)
y_final2=[]
print(model.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

prediccion['sepostulo']=y_final2
prediccion.head()


[0 1]
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,0.0,7.0,1.0,3.0,20.0,20.0,0.089120
1,1,739260,6v1xdL,31.0,False,True,False,False,False,False,...,0.0,0.0,False,3.0,30.0,1.0,0.0,3.0,3.0,0.077955
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,1.0,6.0,0.0,1.0,6.0,6.0,0.006203
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,3.0,1.0,0.0,0.0,0.0,0.001731
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,1.0,4.0,0.0,0.0,1.0,1.0,0.000849


In [55]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.4928002963126386


,id,sepostulo
0,0,0.089120
1,1,0.077955
2,2,0.006203
3,3,0.001731
4,4,0.000849


In [56]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/xgbmean7.csv",index=False)